# Cyclistic Case Study

## Case Background

As a junior data analyst working in the marketing analyst team at Cyclistic, a bike-sharing company active in Chicago, IL, my team has been tasked with understanding how casual riders and annual members use Cyclistic bikes differently. Casual riders consist of customers that purchase single-ride or full-day passes, whereas annual members subscribe yearly for unlimited biking access. The marketing director postulates that the company's future success depends on maximizing the number of yearly memberships by converting casual riders into annual members. My team will be designing a new marketing strategy which can best execute this idea with compelling data insights and professional visualizations.


### Quality assessment

*Reliablity: ?*

There are a lot of missing data values, but as you'll see in the data cleaning section below, we've found a way to stay in the 95% confidence range for the most part.

*Original: ✔*

Data is from a first party source.

*Comprehensive: ✔*

There are enough fields to paint a full picture of ridership statistics between members and casual riders.

*Current: ✔*

Data is from 2023, only one year prior to this study.

*Vetted: ✔*

This process is explore below, but there seem not to be too high a degree of extreme error. 

## Approach (at a glance)

The 6 steps to effective data analysis, according to Google, and how I have chosen to execute them for this case study:

### 1.   Ask
Business task: 

Convert casual riders into full members

Questions: 

1) What are the advantages to becoming a member?
2) How do casual/members differ in their use of Cyclistic?
3) Is it truly an effective strategy to convert them?

### 2.   Prepare
   Data: [divvy-tripdata](https://divvy-tripdata.s3.amazonaws.com/index.html)

   Tools:
   
   1) R Studio for initial glimpse/rough visuals

2) SQL (Google BigQuery) for data cleaning, processing, and analysis

3) Tableau for data visualizations

4) Google Earth for satellite view
    
5) Visual Studio Code/Git for version control

6) Google Slides for [stakeholder presentation](https://docs.google.com/presentation/d/e/2PACX-1vTivq52Psa5nbh7PH1oXxl7W_HKb_zbeqvDixBcz2ClY7gKNysm_ddSaMkkqUzQblHoiUQCZBE7XWgC/pub?start=false&loop=false&delayms=3000)

    
### 3.   Process
1.   Data processing (combining & adding relevant fields)

        |
    ![data set schema](schema_scrnsht-1.png)

        The original datasets each come with 13 unique schema, I will be adding and modifying certain fields to make analysis more streamlined and functional. This was also to make it easier to reference for visualizations.

        |
    ![new data frame schema](new_schema.png)

        The new data frames keep only the most important information, and do not include former fields used to create them. For example, instead of a start_lat and end_lat, there is now just distance traveled in kilometers. However, I can still go back and reference the original data sets if I need the points.

2.   Data analysis (cleaning & analyzing)

        One of the first problems I ran into was the crazy amount of null records, around 25% from each quarter. Initially, I thought to remove them to be able to work with cleaner data, but I decided against that in favor of maintaining data integrity.

3.   Visualization

        I prefer to visualize as I go for a better understanding of trends and behaviors, so I also used R to create rough plots that I would then take a better look into with SQL.

        
        ![types of bikes by membership Q2](q2_ridership_bar_plot.png)

### 4.   Analyze
As I stated before, the majority of analysis for this case study was done in SQL (Google BigQuery), but I did also use R Studio for some initial basic analysis because it's free and doesn't require any cloud service storage. 

The main questions I sought to answer in the Analysis phase were: 

1) How do casual riders and members behave differently?
        This was the main comparison made within the analysis and factored into the majority of queries. Finding patterns and differences between the two groups would be used to answer the remaining question. 
    
2) Which factors seem most important to each group overall?
        This was to see which parts of the business model are doing well already, and which could stand to be imporved overall.

### 5.   Share
Two shareables were created to end this study, the Python markdown you're reading now, and the slides presentation that is a bit broader and less detailed to respect the stakeholders' time. 
There is also a Tableau dashboard for anyone wishing to look further into the visuals.

### 6.   Act
Overall recommendations: 
1) Source more qualitative data
2) Explore different pricing options
3) Highlight membership perks

# Initial Processing (& notes)

### Combining Data:

As the analyst of this team, my first step is to aggregate all data into meaningful sets. I've chosen quarterly and an annual data set both so that everything is organized, uniform, and verifiable. Working with the smaller quarterly data sets also requires less computational power, which is good for me because I'm not working with bleeding edge equipment.

In [ ]:
/* 
Cyclistic Case Study: Initial Setup and Quarterly Join Queries
Q1: jan-mar, 639,424 rows
Q2: apr-jun, 1,751,035 rows
Q3: jul-aep, 2,205,714 rows
*/
SELECT 
  *
FROM 
  `cyclistic_data.oct_23`
UNION ALL

SELECT
  *
FROM
  `cyclistic_data.nov_23` 
UNION ALL

SELECT
  *
FROM
  `cyclistic_data.dec_23`;
-- Q4: oct-dec, 1,123,704 rows

In [ ]:
--Annual joins for fully aggregated data set
SELECT
  *
FROM
  `cyclistic_data.q1_data`
UNION ALL

SELECT
  *
FROM
  `cyclistic_data.q2_data`
UNION ALL

SELECT
  *
FROM
  `cyclistic_data.q3_data`
UNION ALL

SELECT
  *
FROM
  `cyclistic_data.q4_data`;

I began with combining some of the more obvious fields, such as start_lat & start_lng into one start_point field. Doing the same with the end_points allowed me to turn 4 columns into 2, and then when creating the working data frame, start_point and end_point were used to calculate distance_km. Referencing certain aspects of ridership, such as trip durations and distance ridden could all be done with the original schema, but it would be difficult to follow along with, hence the more direct schema of the new data frames I've set up.

In [ ]:
--Creating start_point, end_point, and distance fields
--Used Google-specific function ST_GEOGPOINT as BigQuery is not optimized for trig
SELECT
  * EXCEPT(start_lng, start_lat, end_lng, end_lat),
  ROUND(ST_DISTANCE(start_point, end_point)/1000, 2) AS distance
FROM
  (SELECT
    *,
    ST_GEOGPOINT(start_lng, start_lat) AS start_point,
    ST_GEOGPOINT(end_lng, end_lat) AS end_point
  FROM
    `cyclistic_data.apr_23`
  WHERE
    end_lat <> 0)

In [ ]:
--Calculating trip duratiion in seconds and then also formatting time
SELECT 
  *,
  FORMAT_TIMESTAMP("%T", TIMESTAMP_SECONDS(seconds)) as trip_duration
FROM
  (
  SELECT
    *,
    DATE_DIFF(ended_at, started_at, SECOND) AS seconds,
  FROM `cyclistic_data.annual_data` 
  ) 

In [ ]:
--Establishing day of week field 
SELECT
  *,
  CASE 
    WHEN EXTRACT(DAYOFWEEK FROM started_at) = 1 THEN 'Sunday'
    WHEN EXTRACT(DAYOFWEEK FROM started_at) = 2 THEN 'Monday'
    WHEN EXTRACT(DAYOFWEEK FROM started_at) = 3 THEN 'Tuesday'
    WHEN EXTRACT(DAYOFWEEK FROM started_at) = 4 THEN 'Wednesday'
    WHEN EXTRACT(DAYOFWEEK FROM started_at) = 5 THEN 'Thursday'
    WHEN EXTRACT(DAYOFWEEK FROM started_at) = 6 THEN 'Friday'
    WHEN EXTRACT(DAYOFWEEK FROM started_at) = 7 THEN 'Saturday'
  END day_of_week
FROM 
  `cyclistic_data.annual_data`

### Cleaning Data:

Typically, I would remove null values in the processing stage, but if I were to do that with the annual_df I created, it would bring the records from 5,719,877 down to 4,331,707 which is a reduction of nearly 25%. I believe this large of a subset of data could skew the analysis, especially when most of the nulls don't affect the overall processing. For example, the fields with the highest amount of null values are the start/end station names/ids, but there are far fewer nulls in the start/end lat/lng columns, which means that the station fields don't really affect the calculation of the distance and duration fields.

Therefore, I will be reconciling null values on an as-needed basis to better maintain data integrity.

In [ ]:
-- Calculating nulls and zero values
SELECT
  *
FROM 
  `magnetic-energy-424103-m2.cyclistic_data.q1_df`
WHERE
  distance_km IS null
-- distance: 426, duration: 0, end_station: 93,016, ended_at: 0, member: 0
-- only 0.7% of all entries have null distance values, dropping them

In [ ]:
-- Looking for significant outliers in distance & duration fields
SELECT
  member_casual,
  MAX(duration_min) AS max_duration
FROM
  `magnetic-energy-424103-m2.cyclistic_data.q1_df`
GROUP BY
  member_casual
  -- shows max durations of 1,400.9 & 1,400.92
SELECT
  member_casual,
  MIN(duration_min) AS min_duration
FROM
  (SELECT
    member_casual, -- don't want 0 or negative values
    IF(duration_min > 0.0, duration_min, null) AS duration_min
  FROM
    `magnetic-energy-424103-m2.cyclistic_data.q1_df`)
GROUP BY
  member_casual
  --shows mins of 0.02 for both 

In [ ]:
-- Comparing median to max and min < 0 to better understand outliers
SELECT
  DISTINCT median_duration,
  member_casual
FROM
  (SELECT
    ride_id, member_casual, duration_min, distance_km,
    PERCENTILE_DISC(duration_min, 0.5) 
    OVER(PARTITION BY member_casual) AS median_duration
  FROM
    `magnetic-energy-424103-m2.cyclistic_data.q1_df`)
ORDER BY
  median_duration 
  --medians of 7.07 & 8.68 respectively'

### Separate Tables:

The last step of processing I compartmentalized into two separate tables for each quarter and the year: xyz_data & xyz_df. The xyz_data contains all the data from each of its combined parts, albeit more streamlined, whereas the xyz_df is even more specified and thus has fewer fields. 

Each of these data sets were still available for the analysis phase for when I needed to call on more raw data, such as finding the geological boundaries of the area and which types of bikes were being used. 

In [ ]:
--Creating new data frames with most important fields
SELECT
  ride_id, started_at, ended_at, day_of_week, ROUND(seconds/60,2) AS duration_min,
  distance AS distance_km, start_station_name, end_station_name, member_casual
FROM
  `cyclistic_data.q4_data`
ORDER BY
  ride_id DESC

In [ ]:
--Finding geological boundaries
SELECT
  MAX(start_lat) AS north_lmt_1,
  MIN(start_lat) AS south_lmt_1,
  MAX(start_lng) AS east_lmt_1,
  MIN(start_lng) AS west_lmt_1,
  MAX(end_lat) AS north_lmt_2,
  MIN(end_lat) AS south_lmt_2,
  MAX(end_lng) AS east_lmt_2,
  MIN(end_lng) AS west_lmt_2
FROM
  `cyclistic_data.annual_data`


## Analysis

After a very lengthy processing phase, the first thing I wanted to do was look for significant outliers that could skew the analysis. My findings were that just within the intitial unaltered combined data frame, there were 15% of each start_station field, 16% of each end_station field, and 0.4% of end_lat & end_lng all missing for a combined 25% of all records containing null values. Working around these took some doing, but by carefully factoring in which information was needed for each query, I managed to use about 98-99% of all data for analysis.

The first step was figure out how many significant outliers there were and how much they would affect the analysis:

#### Outliers within durations:

In [ ]:
-- Full view of all ride durations to glimpse outliers
SELECT
  ride_id, member_casual, duration_min, distance_km
FROM
  `cyclistic_data.annual_df`
ORDER BY
  duration_min DESC

In [ ]:
--Factoring out trips with durations over one day & 12 hours
SELECT
  ride_id, member_casual, duration_min, distance_km
FROM
  `cyclistic_data.annual_df`
WHERE
  duration_min > 1440 -- 720 for 12h
  AND distance_km > 0
  --AND distance_km IS null
  --AND distance_km = 0
ORDER BY
  duration_min DESC
--1440: 6418 total, 182 > 0, 37 = 0, 6199 null
--720: 9366 total, 2149 > 0, 738 = 0, 6479 null
--Actually I want to know how looks like for several intervals, so I'll make a function query

In [ ]:
--Had to use standard for the function because I decided I wanted to output to look good
#standardSQL
CREATE TEMP FUNCTION NUMFORMAT(number FLOAT64) AS (
  CONCAT(REGEXP_EXTRACT(cast(number as string), r'\d*\.\d{1}'), ' %')
);
SELECT
  total,
    NUMFORMAT(ROUND((total/5719877)*100,1)) AS of_all_rides,
  pos_dist,
    NUMFORMAT(ROUND((pos_dist/total)*100,1)) AS pos_of_total,
  no_dist,
    NUMFORMAT(ROUND((pos_dist/total)*100,1)) AS zeros_of_total,
  null_dist,
    NUMFORMAT(ROUND((pos_dist/total)*100,1)) AS nulls_of_total
FROM  (
  SELECT
    COUNTIF(duration_min > 1440) AS total,
    COUNTIF(duration_min > 1440 AND distance_km > 0) AS pos_dist,
    COUNTIF(duration_min > 1440 AND distance_km = 0) AS no_dist,
    COUNTIF(duration_min > 1440 AND distance_km IS null) AS null_dist,
  FROM  (
    SELECT
      ride_id, member_casual, duration_min, distance_km
    FROM
      `cyclistic_data.annual_df`
  ))
--Replace 1440 with whichever duration value you'd like to view

I ended up wanting to know more about the rates of different outliers at different intervals to know exactly what cut-off point should be used for visualizing.

For durations over 24 hours:
![over_24h](1440_verification.png)

For durations over 6 hours:
![over_6h](360_verification.png)

For durations over 90 minutes:
![over_90min](90_verification.png)

And for durations over 30 minutes:
![over_30min](30_verification.png)

Because analysts want to work within a confidence level range of 95-99%, I decided to work with durations that were below the 45 minute mark for this analysis as 95.8% of all rides were below this interval. This gives us the maximum amount of data to work with while still being at a comfortable confidence level, as you can see here:
![over_45min](45_verification.png)

This shows that of all 5,719,877 rides in 2023, only 240,242 (or 4.2%) had durations longer than 45 minutes. Within that limited scope, 97% of all null distance and 14% of all no distance rides are contained, thus leaving the remaining 95.8% of all data that much more consistent.

#### Outliers within distances traveled:

Next up, I began looking into extreme outliers within the scope of distances traveled by riders. Assuming that only the start point and end point are recorded in a trip, it's entirely possible that riders who maybe ran a quick errand on the bikes and then brought them back to the same station would still have a total of 0km traveled, so I will not be discounting **all** zeroes within the data frame. Instead, I'll be grouping them with the null distances as they both represent trips with either ambiguous or missing data.

Next, I'll be identifying the trips that were definitely possible by calculating the rates of travel by riders. According to several internet bicycle hobbiest forums, the maximum velocity of a biker who is not a professional would be around 32km/ph. Therefore, the possible_trips column will contain all trips with rates above 0km/ph and below 32km/ph as these would all definitely be acheivable for most people. 

Impossible trips will be those in which too vast of a distance was covered in too short of time, as this is likely the result of an error.

In [ ]:
--Identifying extreme outliers by distance traveled
#standardSQL
CREATE TEMP FUNCTION NUMFORMAT(number FLOAT64) AS (
  CONCAT(REGEXP_EXTRACT(cast(number as string), r'\d*\.\d{1}'), ' %')
);
SELECT
  subscriber_type,
  total_trips,
    NUMFORMAT(ROUND((total_trips/5719877)*100,1)) AS of_all_rides,
  possible_trips,
    NUMFORMAT(ROUND((possible_trips/total_trips)*100,1)) AS pos_of_total,
  impossible_trips,
    NUMFORMAT(ROUND((impossible_trips/total_trips)*100,1)) AS impos_of_total,
  ambiguous_trips,
    NUMFORMAT(ROUND((ambiguous_trips/total_trips)*100,1)) AS ambigs_of_total,
  null_trips,
    NUMFORMAT(ROUND((null_trips/total_trips)*100,1)) AS nulls_of_total
FROM  (
  SELECT
    member_casual AS subscriber_type,
    COUNT(ride_id) AS total_trips,
    COUNTIF(km/ph > 0 AND km/ph < 32) AS possible_trips,
    COUNTIF(km/ph >= 32) AS impossible_trips,
    COUNTIF(km/ph = 0) AS ambiguous_trips,
    COUNTIF(km/ph IS null) AS null_trips,
  FROM  (
    SELECT
      ride_id, member_casual, 
      distance_km AS km, 
      duration_min/60 AS ph
    FROM
      `cyclistic_data.annual_df`
    WHERE
      duration_min >= 1
      AND duration_min IS NOT null
    ORDER BY
      ph ASC
  )
  GROUP BY 
    member_casual
  ) 
  

I also broke the results down by membership so that we could start comparing behaviors of casual riders and members:

![dist_outliers](verify_dist_outliers-1.png)

There are null values for some of the percentage columns because the amount is too far below 0.1% to round up. Regardless, though, instead of losing ~25% of all data by dropping all records with nulls, we're getting 98.4% useable data, of which we're only losing ~0.3% for sure, and needing further investigation on another 9%. All in all, 91% of original data isn't the best number to walk away with, but it's better than 75%.

#### Comparing casual riders to members:

First, we'll look at a basic breakdown of ridership percentages between members and casual riders to understand their standing:

In [ ]:
-- Basic table to see a numeric breakdown of ridership
SELECT
  total_trips, member_trips, casual_trips,
  ROUND(member_trips/total_trips, 2)*100 AS member_percentage,
  ROUND(casual_trips/total_trips, 2)*100 AS casual_percentage
FROM (
  SELECT
    COUNT(ride_id) AS total_trips,
    COUNTIF(member_casual = 'member') AS member_trips,
    COUNTIF(member_casual = 'casual') AS casual_trips
  FROM
    `cyclistic_data.annual_df`
  );
-- 64% of all trips are by members

In [ ]:
-- Viewing the distribution of rides by quarter
SELECT
  COUNT(q1) AS q1_rides,
  COUNT(q2) AS q2_rides,
  COUNT(q3) AS q3_rides,
  COUNT(q4) AS q4_rides
FROM (
  SELECT
    IF(month <= 3, ride_id, null) AS q1,
    IF(month <= 6 AND month > 3, ride_id, null) AS q2,
    IF(month <= 9 AND month > 6, ride_id, null) AS q3,
    IF(month <= 12 AND month > 9, ride_id, null) AS q4
  FROM (
    SELECT 
      ride_id,
      EXTRACT(MONTH FROM started_at) AS month
    FROM 
      `cyclistic_data.annual_df` 
  )
)
-- shows 639,424 for q1, 1751035 for q2, 2,205,714 for q3, and 1,123,704 for q4

![distribution](ride_distribution_quarter.png)

(I ended up having to plot this in R because Tableau Public has a dual axis limit)

In [ ]:
-- A more detailed view of the above query with weekday breakdown
#standardSQL
CREATE TEMP FUNCTION NUMFORMAT(number FLOAT64) AS (
  CONCAT(REGEXP_EXTRACT(cast(number as string), r'\d*\.\d{1}'), ' %')
);
SELECT
  day_of_week,
  total_trips, member_trips,
  casual_trips, --trip_date, 
  NUMFORMAT(ROUND(member_trips/total_trips,5)*100) AS member_percentage,
  NUMFORMAT(ROUND(casual_trips/total_trips,5)*100) AS casual_percentage
FROM  (
  SELECT
    COUNT(ride_id) AS total_trips,
    COUNTIF(member_casual='member') AS member_trips,
    COUNTIF(member_casual='casual') AS casual_trips,
    --DATE(started_at) AS trip_date,
    day_of_week
  FROM
    `cyclistic_data.annual_df`
  WHERE
    ride_id = ride_id
  GROUP BY
    day_of_week
    --trip_date
    )
ORDER BY
  total_trips DESC
--We can see that during the week, it's about an 80:20 split to the members,
--but on the weekends, that ratio leans toward a 70:30 split

![rides_by_day](rides_by_day_member-1.png)

We've learned quite a bit just from asking "who rides more?" and "who rides when?". The next step is to visualize this data so that we can have a better representation of how it all looks beyond just numbers. A quick bar chart should be more than sufficient for showing our stakeholders.

![by_day](<Dashboard 1.png>)

While members account for about 77% of total rides, they are actually closer to 80% during the week, and then slide back to about 30% during the weekend. This shows that there is a higher amount of casual riders during the weekend than during the week. How about what times they tend to use Cyclistic most?

In [ ]:
-- Looking at who rides when
#standardSQL
CREATE TEMP FUNCTION NUMFORMAT(number FLOAT64) AS (
  CONCAT(REGEXP_EXTRACT(cast(number as string), r'\d*\.\d{1}'), ' %')
);
SELECT
  start_hour,
  --day_of_week,
  --trip_date, 
  total_trips, member_trips, casual_trips, 
  NUMFORMAT(ROUND(member_trips/total_trips,5)*100) AS member_percentage,
  NUMFORMAT(ROUND(casual_trips/total_trips,5)*100) AS casual_percentage
FROM  (
  SELECT
    --DATE(started_at) AS trip_date,
    --day_of_week,
    EXTRACT(HOUR FROM started_at) AS start_hour,
    COUNT(ride_id) AS total_trips,
    COUNTIF(member_casual='member') AS member_trips,
    COUNTIF(member_casual='casual') AS casual_trips    
  FROM
    `cyclistic_data.annual_df`
  WHERE
    ride_id = ride_id
  GROUP BY
    --day_of_week,
    --trip_date,
    start_hour)
ORDER BY
  total_trips DESC
-- Here we can actually see that while members outnumber casual riders all day long,
-- There are actually more casuals between midnight and 4:00am

![by_hour](by_hour-1.png)

Here we can see that out of the total trips by hour that while members outnumber casual riders all day long, there are actually more casuals between midnight and 4:00am. This let's us know that people may see Cyclistic as an affordable, available, and more widely effective method of late night urban transport. It actually makes perfect sense in an urban center like Chicago, where rideshare apps are very expensive, public transport ends around 10pm, and walking is probably not a safe bet.

Next, we'll check out some averages in trip durations and distances between weekdays, rideable types, and month.

In [ ]:
-- Looking at ridership stats (distance + duration) of the 3 bike types 
SELECT
  rideable_type,
  ROUND(AVG(seconds)/60,2) AS avg_duration,
  ROUND(AVG(distance),2) AS avg_distance
FROM 
  `cyclistic_data.q1_data` 
WHERE
  seconds < 2700
  --the working limit of trip durations (<45min)
GROUP BY
  1
LIMIT 9
--data consistent with other avg ridership stats except for one thing:
--the returned table showed a ridiculously high avg duration on docked_bike
--which is only used by casual riders, so I queried just those

![bike_types](year_ridership_bar_plot.png)

In [ ]:
-- Looking at ridership distance stats by membership type along day of week
SELECT
    day_of_week,
    ROUND(AVG(distance_km),2) AS avg_ride_distance,
    ROUND(AVG(members),2) AS avg_member_dist,
    ROUND(AVG(casuals),2) AS avg_casual_dist
FROM
    (SELECT
      member_casual, day_of_week, distance_km, 
      IF(member_casual = 'member', distance_km, null) AS members,
      IF(member_casual = 'casual', distance_km, null) AS casuals
    FROM
    `cyclistic_data.q1_df`)
GROUP BY
   day_of_week
ORDER BY
  avg_ride_distance DESC
--Seems that Sundays are the most popular and Fridays are the least for everyone

![avg_dist](avg_dist-2.png)

In [ ]:
-- And the same query with average trip durations 
SELECT
    day_of_week,
    ROUND(AVG(duration_min),2) AS avg_ride_duration,
    ROUND(AVG(members),2) AS avg_member_drtn,
    ROUND(AVG(casuals),2) AS avg_casual_drtn
FROM
    (SELECT
      member_casual, day_of_week, duration_min, 
      IF(member_casual = 'member', duration_min, null) AS members,
      IF(member_casual = 'casual', duration_min, null) AS casuals
    FROM
    `cyclistic_data.q1_df`)
GROUP BY
   day_of_week
ORDER BY
  day_of_week

![drtn_avg_med](cllag.png)

## Conclusions

From our detailed analysis of provided data, we have been able to see the differences and similarities in behaviors of both casual riders and members. Overall, there are far more similarities than stark contrasts in member and casual rider tendencies, however there were some key divergences.

#### For similarities:

Both riders tended to follow the same trends as far as standard distribution of rides along a 24hr scale, 7 day scale, and quarterly/seasonal scale. They also both tend to average trips around the same distances and durations, with casual members taking marginally longer to travel a negleiibly further distance per trip. 

They also both seem to favor electic bikes slightly more than classic bikes, and each appear to favor the same locations in the downtown area as we can see below.

In [ ]:
--Looking at the top 10 most popular station combos between members & casuals
SELECT
    start_station_name, end_station_name,
    COUNT(*) AS combo,
    COUNTIF(member_casual='member') AS member_combo,
    COUNTIF(member_casual='casual') AS casual_combo
FROM 
    `cyclistic_data.annual_df`
WHERE
    start_station_name = start_station_name
    AND end_station_name = end_station_name
    AND end_station_name IS NOT null
    AND start_station_name IS NOT null
GROUP BY
    1,2
ORDER BY
    combo DESC
LIMIT
    10

![top_stations](station_combos.png)

![map](cyclistic_range.png)

#### For differences:

Within the standard distributions for rides taken between members and casual riders, two of the biggest differences are that casual riders tend to prefer weekends and late night rides somewhat more than members. This could be due to any number of reasons, such as: events in downtown, response to rideshare surge rates, lack of public transit after hours, or any number of other environmental factors.

Either way, doing some more research into this difference could yield more exact data with which we could analyze and extrapolate a definitive reason. With the intent of converting more casual rides to members, this would be the area in which I focus my next analysis.

The only other major difference between the two groups was found in the preference of rideable type. It seems that docked bikes are either only available to casual riders, or 100% ignored by members, but either way, they only make up 2.8% of all rides taken by casual riders. It's worth looking into if for nothing else than to better understand customer tendencies, but not a huge factor in this study.

## Recommendations:

#### *A move toward qualitative data*

As it stands, all of the data analyzed thus far has been entirely quanititative, which means that there is a fair amount of guesswork to be done in deciphering customer behavior and desires. These data sets have been useful for identifying significant outliers (due to human or technical error), finding averages in ridership and ride data, and locating popular customer hubs.

However, being able to qualify these data with context could tell us a lot more about what makes Cyclistic attractive to customers and how we could improve the experience. For example: including error reports with data sets could inform us of a recurring issue with our devices or their ease of use. I will give several other examples below with the top three next steps for this company.

#### *1) Customer surveys*

Finding out who our customers are and why they use our app will help us better understand user demographics. This will allow us to better cater towards specific groups, which in turn will help to draw in more people in that area. The surveys don't need to be extensive, just enough to give us age, occupation, and reason for interest. If we want to get an even better understanding and recieve more reviews, we could offer something like "the next ride free!" in exchange for a review on the app. Each of these would be quick and easy ways to make our future data more qualitative.

#### *2) Alternative pricing models*

With step one completed, we could start being more proactive with our marketing instead of responsive. Once we understand their interests better, we may find that membership is just not that attractive of an offer. In this case, we should be open to looking to alternative pricing options.

If, for example, we find that a high percentage of casual riders use our services for local downtown events as a way to avoid parking and rideshare costs, we could offer a competetively priced weekend package. This saves the customer money, yields more revenue than just one or two rides, and could potentially become an effective word-of-mouth marketing opportunity. 

#### *3) Differentiating membership experience*

Given just the quantitative data we've explored thus far, there doesn't seem to be much of a reason to become a member outside of a potentially lower pricepoint. For local commuters within the average distance curve, this is probably a good enough reason to save a little bit of money, but for anyone else, the offer may not seem worth it. 

Highlighting the perks of membership within and outside of the app would be a good way to get riders to at least look into getting a membership. Maybe members' top stations and locations are always stocked during surge times during the day, or maybe they have access to an exclusive bike model that's faster or more comfortable. Regardless of what perks members get, it's crucial that we display the difference for our riders so that they have more of a reason to explore membership. 